# Genre Clustering with data from Spotify API using KNN Algorithms (Hip-hop vs Jazz)
#### Claire (Lapatrada) Jaroonjetjumnong

## Part 1: Data gathering and cleaning with Spotify API

In [11]:
# find a GOOD dataset
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

OAuth = SpotifyOAuth(
        scope = "user-library-read",         
        redirect_uri ='http://localhost:8888/callback',
        client_id = 'd721dd9969c5408f8894a7d93c904596',
        client_secret = '1ba0721de55245a9b4e88a11e3428930',
        username = '215s7y5cw3g4sjiizykwdgcty')
sp = spotipy.Spotify(auth_manager=OAuth)

PLAYLIST_ID = '37i9dQZF1DXbkfWVLd8wE3' # hiphop 90
PLAYLIST_ID2 = '37i9dQZF1DX4wta20PHgwo' # jazz 100

offset = 0
songs = []
items = []
ids = []

while True:
    content = sp.user_playlist_tracks(user = 'spotify', playlist_id = PLAYLIST_ID)
    songs += content['items']
    if content['next'] is not None:
        offset += 100
    else:
        break
        
while True:
    content = sp.user_playlist_tracks(user = 'spotify', playlist_id = PLAYLIST_ID2)
    songs += content['items']
    if content['next'] is not None:
        offset += 100
    else:
        break

### Part 1.1: Data frame and Normalizing values


In [12]:
import pandas as pd

index = 0
audio_features = []
features_list = []
ids = []
names = []
for song in songs[:180]:
    ids.append(song['track']['id'])
    names.append(song['track']['name'])

while index < len(ids):
    audio_features += sp.audio_features(ids[index:index + 50])
    index += 50

for features in audio_features:
    features_list.append([features['energy'], features['danceability'],
                            features['liveness'], features['tempo'],
                            features['speechiness'], features['valence'],
                            features['loudness'], features['instrumentalness'],
                            features['acousticness'], features['key']])

df = pd.DataFrame(features_list, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])

### Normalizing values
from sklearn import preprocessing
x = df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])

df["genre"] = 'n'
df["genre"][:89] = 'hip'
df["genre"][89:] = 'jazz'
df['name'] = names
df

/var/folders/j6/14l5ybgs2cz_vt6rg313741w0000gp/T/ipykernel_24660/3656032408.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["genre"][:89] = 'hip'
/var/folders/j6/14l5ybgs2cz_vt6rg313741w0000gp/T/ipykernel_24660/3656032408.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["genre"][89:] = 'jazz'


,energy,danceability,liveness,tempo,speechiness,valence,loudness,instrumentalness,acousticness,key,genre,name
0,0.646750,0.534508,0.418659,0.268852,0.021254,0.422289,0.824287,0.000000,0.038198,0.545455,hip,Flashing Lights
1,0.871167,0.941263,0.057431,0.299737,0.433923,0.590429,0.933910,0.000000,0.039506,0.636364,hip,Hit 'Em Up - Single Version
2,0.847271,0.578561,0.138537,0.297802,0.547559,0.696055,0.921676,0.000000,0.001694,1.000000,hip,"Nuthin' But A ""G"" Thang"
3,0.659217,0.660793,0.082278,0.764440,0.635943,0.945031,0.814359,0.000000,0.028137,0.454545,hip,"So Fresh, So Clean"
4,0.871167,0.503671,0.040553,0.229026,0.656987,0.710067,0.931360,0.000131,0.017773,0.090909,hip,Izzo (H.O.V.A.)
...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.133497,0.383260,0.350680,0.407578,0.008207,0.163613,0.427322,0.966771,0.940636,1.000000,jazz,Links
176,0.125186,0.060206,0.129161,0.184680,0.023148,0.102177,0.550486,0.764278,0.981889,0.545455,jazz,A Flower Is a Lovesome Thing
177,0.075315,0.380323,0.092827,0.549046,0.092803,0.158224,0.416393,0.912773,1.000000,0.000000,jazz,Answer Me - Live from Budapest
178,0.234278,0.246696,0.099859,0.431614,0.019992,0.214270,0.551943,1.000000,0.866179,0.000000,jazz,Just Go Ahead


### Part 1.2: Audio Features Correlation Heatmap

In [5]:
import seaborn as sns
corr = df.corr()
sns.heatmap(corr, annot=True)

ModuleNotFoundError: No module named 'seaborn'

## Part 2: Train and test the model

### Part 2.1: Splitting train, validation, and test set

In [6]:
input_data = df[["energy", "danceability", "liveness", "tempo", "speechiness", "valence", "loudness", "instrumentalness", "acousticness", "key"]]
target = df["genre"]

In [68]:
training_data = pd.concat([df.iloc[:75, :], df.iloc[89:115, :]])
test_data = pd.concat([df.iloc[75:89, :], df.iloc[115:, :]])

train_data = pd.concat([training_data.iloc[:55, :], training_data.iloc[75:87, :]]) # First
val_data = pd.concat([training_data.iloc[55:75, :], training_data.iloc[87:, :]]) # Remaining

X_train = train_data[["energy", "danceability", "liveness", "tempo", "speechiness", "valence", "loudness", "instrumentalness", "acousticness", "key"]].values
y_train = train_data["genre"].tolist()

X_val = val_data[["energy", "danceability", "liveness", "tempo", "speechiness", "valence", "loudness", "instrumentalness", "acousticness", "key"]].values
y_val = val_data["genre"].tolist()

X_test = test_data[["energy", "danceability", "liveness", "tempo", "speechiness", "valence", "loudness", "instrumentalness", "acousticness", "key"]].values
y_test = test_data["genre"].tolist()


### Part 2.2: Training & prediction

In [115]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

classifier = KNeighborsClassifier(n_neighbors = 3)
classifier.fit(X_train, y_train)

# Use the fitted model to make predictions on the test dataset
test_predictions = classifier.predict(X_test)

print('Model evaluation on the TEST set: \n')
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))
print("TEST accuracy:", accuracy_score(y_test, test_predictions))


Model evaluation on the TEST set: 

[[14  0]
 [ 0 65]]
              precision    recall  f1-score   support

         hip       1.00      1.00      1.00        14
        jazz       1.00      1.00      1.00        65

    accuracy                           1.00        79
   macro avg       1.00      1.00      1.00        79
weighted avg       1.00      1.00      1.00        79

TEST accuracy: 1.0


### Part 2.3: Best K-Values

In [112]:
K_values = [1, 2, 3, 4, 5, 6]

for K in K_values:
    classifier = KNeighborsClassifier(n_neighbors = K)
    classifier.fit(X_train, y_train)
    val_predictions = classifier.predict(X_val)
    print("K=%d, Validation accuracy score: %f" % (K, accuracy_score(y_val, val_predictions)))

K=1, Validation accuracy score: 0.941176
K=2, Validation accuracy score: 1.000000
K=3, Validation accuracy score: 1.000000
K=4, Validation accuracy score: 1.000000
K=5, Validation accuracy score: 1.000000
K=6, Validation accuracy score: 1.000000


## Part 3: Predict new data

### Part 3.1: Loading and cleaning new data

In [102]:
# Download test data
jazz = '37i9dQZF1DWTALrdBtcXjw' # 100
hh = '37i9dQZF1DX9XeSNHcmQdR' # 100

offset2 = 0
songs2 = []
items2 = []
ids2 = []

offset3 = 0
songs3 = []
items3 = []
ids3 = []

while True:
    content2 = sp.user_playlist_tracks(user = 'spotify', playlist_id = jazz)
    songs2 += content2['items']
    if content2['next'] is not None:
        offset2 += 100
    else:
        break
        
while True:
    content3 = sp.user_playlist_tracks(user = 'spotify', playlist_id = hh)
    songs3 += content3['items']
    if content3['next'] is not None:
        offset3 += 100
    else:
        break

index = 0
audio_features = []
features_list = []
ids = []
names = []

for song in songs2:
    ids.append(song['track']['id'])
    names.append(song['track']['name'])

while index < len(ids):
    audio_features += sp.audio_features(ids[index:index + 50])
    index += 50

for features in audio_features:
    features_list.append([features['energy'], features['danceability'],
                            features['liveness'], features['tempo'],
                            features['speechiness'], features['valence'],
                            features['loudness'], features['instrumentalness'],
                            features['acousticness'], features['key']])

jazz_p = pd.DataFrame(features_list, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])

### Normalizing values
x = jazz_p.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
jazz_p = pd.DataFrame(x_scaled, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])
jazz_p['name'] = names
jazz_p['genre'] = "jazz"

index = 0
audio_features = []
features_list = []
ids = []
names = []

for song in songs3:
    ids.append(song['track']['id'])
    names.append(song['track']['name'])

while index < len(ids):
    audio_features += sp.audio_features(ids[index:index + 50])
    index += 50

for features in audio_features:
    features_list.append([features['energy'], features['danceability'],
                            features['liveness'], features['tempo'],
                            features['speechiness'], features['valence'],
                            features['loudness'], features['instrumentalness'],
                            features['acousticness'], features['key']])

hip_p = pd.DataFrame(features_list, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])

### Normalizing values
x = hip_p.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
hip_p = pd.DataFrame(x_scaled, columns=['energy', 'danceability', 
                                        'liveness', 'tempo',
                                        'speechiness', 'valence',
                                        'loudness', 'instrumentalness',
                                        'acousticness', 'key'])

hip_p['name'] = names
hip_p['genre'] = "hip"


### Part 3.2: Predicting hip-hop playlist

In [120]:
count = 0
for i in range(100):

    arr = hip_p.iloc[i, :-2].values

    # Reshaping into 2D
    arr = arr.reshape(1, -1)

    test_predictions = classifier.predict(arr)
    if test_predictions[0] == hip_p.iloc[i, -1]:
        count += 1
    print(f"Prediction: {test_predictions[0]}, True: {hip_p.iloc[i, -1]}, Song: {hip_p.iloc[i, -2]}")
print(count/100)

Prediction: hip, True: hip, Song: It Was A Good Day
Prediction: hip, True: hip, Song: Stan
Prediction: hip, True: hip, Song: Insane in the Brain
Prediction: hip, True: hip, Song: Big Poppa - 2007 Remaster
Prediction: hip, True: hip, Song: Doin' Our Own Dang
Prediction: hip, True: hip, Song: Can I Kick It?
Prediction: hip, True: hip, Song: Through The Wire
Prediction: hip, True: hip, Song: My Name Is
Prediction: jazz, True: hip, Song: Self Care
Prediction: hip, True: hip, Song: Money Trees
Prediction: hip, True: hip, Song: Doo Wop (That Thing)
Prediction: hip, True: hip, Song: Ms. Jackson
Prediction: hip, True: hip, Song: Hurricane
Prediction: hip, True: hip, Song: Gettin' Jiggy Wit It
Prediction: hip, True: hip, Song: Keep Their Heads Ringin'
Prediction: hip, True: hip, Song: Got 'Til It's Gone
Prediction: hip, True: hip, Song: Gangsta's Paradise
Prediction: hip, True: hip, Song: Without Me
Prediction: hip, True: hip, Song: Praise God
Prediction: hip, True: hip, Song: Hypnotize - 2007 